<a href="https://colab.research.google.com/github/kowoonho/machine_learning_project/blob/main/traffic_classify_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#교통 표지판 분류 프로젝트


import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle_data/"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/My Drive/kaggle_data

/content/gdrive/My Drive/kaggle_data


In [ ]:
import pickle

with open("/content/gdrive/My Drive/kaggle_data/data0.pickle", "rb") as fr:
  data = pickle.load(fr)

In [ ]:
print(type(data))

<class 'dict'>


In [ ]:
x_train = data['x_train'][:50000]/255.0
y_train = data['y_train'][:50000]

images_and_labels = list(zip(x_train, y_train))
y_train.max()

42

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(1)


In [ ]:
x_train = torch.as_tensor(x_train).float().to(device)
y_train = torch.as_tensor(y_train).to(device)



In [ ]:
images_and_labels = list(zip(x_train, y_train))


In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=images_and_labels, batch_size=100, shuffle=True)

In [ ]:

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 32, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.fc1 = nn.Linear(128*4*4, 625, bias=True)
    nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(0.5)
    )
    self.fc2 = nn.Linear(625, 43, bias = True)
    nn.init.xavier_uniform_(self.fc2.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

model = CNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)


In [ ]:
total_batch = len(data_loader)
print(total_batch)

500


In [ ]:
for epoch in range(10):
  avg_cost = 0
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    hypothesis = model(x)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y.unsqueeze(1), 1)
    optimizer.zero_grad()
    cost = criterion(hypothesis, y_one_hot)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  
  print("Epoch : {:2d} Cost : {:.6f}".format(
      epoch+1, cost.item()
  ))

    

Epoch :  1 Cost : 0.629131
Epoch :  2 Cost : 0.184229
Epoch :  3 Cost : 0.038964
Epoch :  4 Cost : 0.094866
Epoch :  5 Cost : 0.018898
Epoch :  6 Cost : 0.042897
Epoch :  7 Cost : 0.066277
Epoch :  8 Cost : 0.043124
Epoch :  9 Cost : 0.044282
Epoch : 10 Cost : 0.015214


In [ ]:
print(data.keys())

dict_keys(['x_test', 'y_validation', 'x_validation', 'labels', 'x_train', 'y_test', 'y_train'])


In [ ]:
with torch.no_grad():
  x_test = (torch.as_tensor(data['x_test']).float()/255.0).to(device)
  y_test = torch.as_tensor(data['y_test']).to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy : ",accuracy.item())  


Accuracy :  0.895882785320282
